In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [32]:
# load ascii text and covert to lowercase
filename = "DanceScripts.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [33]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [34]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  96089
Total Vocab:  33


In [35]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  95989


In [36]:
...
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

In [37]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [38]:
...
# define the checkpoint
filepath="weights-improvement-21-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

filename = "./weights-improvement-08-0.3936.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [39]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20


2024-03-29 14:43:33.134498: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-29 14:43:33.288829: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  3/750 [..............................] - ETA: 27s - loss: 0.3483  

2024-03-29 14:43:33.443033: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


750/750 [==============================] - ETA: 0s - loss: 0.3990
Epoch 1: loss improved from inf to 0.39899, saving model to weights-improvement-21-01-0.3990.hdf5
750/750 [==============================] - 26s 33ms/step - loss: 0.3990
Epoch 2/20
750/750 [==============================] - ETA: 0s - loss: 2.7473
Epoch 2: loss did not improve from 0.39899
750/750 [==============================] - 25s 33ms/step - loss: 2.7473
Epoch 3/20
749/750 [============================>.] - ETA: 0s - loss: 3.1083
Epoch 3: loss did not improve from 0.39899
750/750 [==============================] - 24s 33ms/step - loss: 3.1082
Epoch 4/20
749/750 [============================>.] - ETA: 0s - loss: 2.9402
Epoch 4: loss did not improve from 0.39899
750/750 [==============================] - 24s 32ms/step - loss: 2.9400
Epoch 5/20
749/750 [============================>.] - ETA: 0s - loss: 2.9465
Epoch 5: loss did not improve from 0.39899
750/750 [==============================] - 24s 32ms/step - loss: 2.9

In [43]:
import sys

int_to_char = dict((i, c) for i, c in enumerate(chars))
seed = "welcome "
seed = " " * (seq_length - len(seed)) + seed
pattern = [char_to_int[char] for char in seed]
print("Seed:", pattern)
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
 x = np.reshape(pattern, (1, len(pattern), 1))
 x = x / float(n_vocab)
 prediction = model.predict(x, verbose=0)
 index = np.argmax(prediction)
 result = int_to_char[index]
 seq_in = [int_to_char[value] for value in pattern]
 sys.stdout.write(result)
 pattern.append(index)
 pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 29, 11, 18, 9, 21, 19, 11, 1]
"                                                                                             welcome  "
tturise phetii to the siglesiag 
pet's seleome soe soage and eancst 

poepare to be daztled by the radiant charm of luna! with her infectious energy and captivating soage presence, she'll lrane you sheolbo tiee woan bedce  she'll take you on a journey tfrough the masie 

erace yourselves for the explosive energy of blaze! with his fiery passion and eantivating srage presence, she'll lrane you sheolbo tie  with  with his siart oo ce cadtis ti  hatie st the moage with a performance that iesveset t
Done.
